In [1]:
import numpy as np
import pandas as pd
import random
import math
import collections
from sklearn import preprocessing

In [2]:
#Read txt file
data= pd.read_csv("/Users/SKU/Desktop/fall2017/data mining/Project 3/project3_dataset2.txt", sep="\t", header=None)


In [3]:
# Separate Attribute and Label
labels =data[data.columns[-1]]
data = data.ix[:, :len(data.columns)-2]


/Users/SKU/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Handeling Categorical Data

#Select Categorical Columns from the dataframe
obj_df = data.select_dtypes(include=['object']).copy()

#One-Hot Encoding for categorical data
for idx in obj_df.columns:
    obj_df= pd.get_dummies(obj_df,columns=[idx],drop_first=False)

obj_dataset = obj_df.values.tolist()


In [5]:
# Handeling Numerical Data
#Select numerical data from the data
num_df = data.select_dtypes(include=['int64','float64']).copy()

# Normalize Numeric Data
if (len(num_df.columns) !=0):
    num_dataset = preprocessing.normalize(num_df, norm='l2')
    num_dataset
else:
    num_dataset = []
print(len(num_dataset))

462


In [6]:
# Dataset Format to be used for algorithm
if len(num_dataset)!= 0:
    dataset = np.column_stack((num_dataset, obj_dataset))
    dataset = np.column_stack((dataset, labels))
else:
    dataset = np.column_stack((obj_dataset, labels))
(len(dataset[0]))

11

In [7]:
# get all labels from the dataset
labels = set(col[-1] for col in dataset)
labels = list(labels)


In [8]:
#Set Values
label_col = len(dataset[0])
n_fold = 10
tree_count = 5
max_depth = 6

In [9]:
# assign initial weight to all record 
def initial_weights(train_dataset):
    n_rows = len(train_dataset)
    n_col = len(train_dataset[0])
    wt = 1/n_rows
    wt_list = []
    for i in range(0, n_rows):
        wt_list.append(wt)
    weight_record[0] = wt_list
    
    #as all the train set has same weight randomle select any row
    no_sample = math.ceil(0.63* len(train_dataset))
    
    return no_sample
    #return train_dataset, datasets[1], no_sample

In [10]:
# select boosting sample
def boost_sample(no_sample, train_data, wt_list):
    
    index_list = np.argpartition(wt_list, -no_sample)[-no_sample:]
    samples = []

    for sam in index_list:
        samples.append(train_data[sam])
        
    return samples

In [11]:
#base clasifier code - to build decision tree
def split(idx, val, dataset):
    lft, rt = list(), list()
    for row in dataset:
        if row[idx] < val:
            lft.append(row)
        else:
            rt.append(row)
    return lft, rt   

# Calculate the Gini index for splitted dataset
def gini_idx(attr_grps):
    # count all possible samples at split point
    total_instances = float(sum([len(attr) for attr in attr_grps]))
    #sum weighted Gini index for each group
    gini = 0.0
    for attr in attr_grps:
        size = float(len(attr))
        
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        
        # score the group based on the score for each class
        for class_val in labels:
            p = [row[-1] for row in attr].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / total_instances)
    return gini


# Select the best split point for a dataset with different attributes
def get_split(dataset1, attr_list):
    b_idx, b_val, b_gini_score, b_attr_grp = 9999, 9999, 9999, None
    for idx in range(len(dataset1[0])-1):
        
        #Use only the attributes not used as nodes
        if idx not in attr_list:
            for row in dataset1:
                attr_grps = split(idx, row[idx], dataset1) 
                gini = gini_idx(attr_grps)
                if gini < b_gini_score:
                    b_idx, b_val, b_gini_score, b_attr_grp = idx, row[idx], gini, attr_grps
    return {'idx':b_idx, 'a_val':b_val, 'grps':b_attr_grp, 'gini':  b_gini_score}

In [12]:
def terminal(attr_grp):
    results = [row[-1] for row in attr_grp]
    return max(set(results), key=results.count)

# Create child splits or terminal nodes
def node_split(node, depth, attr_list):
    lft, rt = node['grps']
    del(node['grps'])
    
    # check for a no split
    if not lft or not rt:
        node['lft'] = node['rt'] = terminal(lft + rt)
        return

    # process only left child
    if len(attr_list) == len(lft[0])-1:
        node['lft'] = terminal(lft)
    else:
        node['lft'] = get_split(lft, attr_list)
        attr_list.append(node['lft']['idx'])
        node_split(node['lft'], depth+1,attr_list)
        
    # process only right child
    if len(attr_list) == len(rt[0])-1: 
        node['rt'] = terminal(rt)
    else:
        node['rt'] = get_split(rt,attr_list)
        attr_list.append(node['rt']['idx'])
        node_split(node['rt'], depth+1, attr_list)
   
        
# Build a decision tree
def build_tree(train_dataset, attr_list):
    root = get_split(train_dataset, attr_list)
    attr_list.append(root['idx'])
    
    node_split(root, 1,attr_list)
    return root, attr_list

In [13]:
#Decision Tree Classification Algorithm
def decision_tree(train_dataset, test_dataset, attr_list):
    tree, attr = build_tree(train_dataset, attr_list)
    predictions = list()
    for row in test_dataset:
        predicts = predict(tree, row)
        predictions.append(predicts)
    return predictions, tree

In [14]:
# Prediction with the tree
def predict(node, row):
    if row[node['idx']] < node['a_val']:
        if isinstance(node['lft'], dict):
            return predict(node['lft'], row)
        else:
            return node['lft']
    else:
        if isinstance(node['rt'], dict):
            return predict(node['rt'], row)
        else:
            return node['rt']

In [15]:
def compare_results(boost_result, train_data):
    actual_labels = list(row[-1] for row in train_data)
    diff_indices=[]
    for i in range(0, len(actual_labels)):
        if actual_labels[i] != boost_result[i]:
            diff_indices.append(i)
    return diff_indices

In [16]:
def classifier_error(diff_indices, train_len, last_assigned_wt):
    numerator = 0
    denominator = 0
    for i in range(0, train_len):
        if i in diff_indices:
            numerator = numerator + last_assigned_wt[i]
        denominator = denominator + last_assigned_wt[i]
    epsilon = numerator / denominator
    alpha = math.log((1-epsilon)/epsilon)* 0.5
    return epsilon, alpha       

In [17]:
def reassign_weights(index_list, train_length, last_assigned_wt):
    classified_data = classifier_error(index_list, train_length, last_assigned_wt)
    alpha = classified_data[1]
    new_weights = []
    for i in range(0, train_length):
        weight =0
        if i in index_list:
            weight = last_assigned_wt[i] * math.exp(alpha)
        else :
            weight = last_assigned_wt[i] * math.exp(-alpha)
        new_weights.append(weight)
    
    normalize_wt=[]
    sum_all = sum(new_weights)
    for i in range(0, len(new_weights)):
        normalize_wt.append(new_weights[i]/sum_all)
        
    return normalize_wt, classified_data[0], classified_data[1]    

In [18]:
# divide entire data into n folds
def cross_validation(n_folds):
    updated_datasets = list(dataset)
    size_fold = math.floor(len(updated_datasets)/n_folds)
    new_dataset = []
    for i in range(0, n_folds):
        count = 0
        fold = []
        while count < size_fold:
            r = random.randint(0, len(updated_datasets)-1)
            fold.append(updated_datasets[r])
            updated_datasets.pop(r)
            count +=1
        new_dataset.append(fold)
    return new_dataset

In [19]:
def get_foldset(updatedset, test_index): 
    foldset = []
    
    #remove the test set and combine other lists for train 
    newset = list(updatedset)
    newset.pop(test_index)
    foldset = sum(newset, [])
 
    return foldset

In [20]:
def calc_results(predicted_list, testdata):
    test_labels = []
    test_labels = [col[-1] for col in testdata]
    tp = 0 
    fn = 0 
    fp = 0 
    tn = 0
    for i in range(0, len(test_labels)):
        if test_labels[i] == 1:
            if predicted_list[i] == 1: 
                tp += 1
            else:
                fn += 1
        else:
            if predicted_list[i] == 1: 
                fp += 1
            else:
                tn += 1
    Accuracy = (tp + tn)/(tp + tn + fp + fn)
    
    if tp!= 0:
        Precision = tp/(tp + fp)
        Recall = tp/(tp + tn)
        F1_measure = (2*tp)/ (2*tp + tn + fp)

    else:
        Precision = 0
        Recall = 0
        F1_measure = 0
    
    return Accuracy, Precision, Recall, F1_measure

In [21]:
# main function
#run k-fold on the train data set
random.seed(1234)
n_fold = 10
validated_dataset = cross_validation(n_fold)
fold_alphas = []
accuracies = []
precisions = []
recalls = []
f1_measures = []
tree_count = 3
max_depth = 6

for i in range(0, n_fold):
    print("----------------beginnning of a fold---------------------")
    test_dataset = validated_dataset[i]
    current_dataset = get_foldset(validated_dataset, i)
    weight_record = collections.OrderedDict()
    #initialize wts and get all data updated weight_record with run 0
    sample_count = initial_weights(current_dataset)
    classifier_output = []
    classifier_errors = []
    classifier_roots = []
    count = 0
    
    while count < tree_count :
        count += 1
        last_assigned = weight_record.popitem()         #last assigned is tuple
        weight_record[last_assigned[0]] = last_assigned[1] 
        last_updated_weights = last_assigned[1]
        #print("after initialization weight record-------------->size of map::" + str(len(weight_record)))
        
        # collect sample for next run by checking weights
        sample_record = boost_sample(sample_count, current_dataset, last_updated_weights)

        attribute_list = []
        
        # create model from collected sample and check all the training sets
        decisions_made = decision_tree(sample_record, current_dataset, attribute_list)
        prediction = decisions_made[0]
        root= decisions_made[1]
        train_length = len(current_dataset)
        
        # compare the predicted results with the original results
        mismatched = compare_results(prediction, current_dataset)
        
        #upadte the weights for each as per to the results from model
        results = reassign_weights(mismatched, train_length, last_updated_weights)
        updated_wts = results[0]
        epsilon = results[1]
        alpha = results[2]
                
        #append all the results into a list
        if epsilon < 0.5:
            classifier_output.append(alpha)
            classifier_errors.append(epsilon)
            key = last_assigned[0] +1
            weight_record[key] = updated_wts
            classifier_roots.append(root)
    
    # calculate result of this fold    
    fold_output = 0
    
    for alphas in classifier_output:
        fold_output += alphas
    
    
    # updated code to check on test data
    predicted_results = []
    for row in test_dataset:
        sum_classifiers = 0
        for i in range(0, len(classifier_output)):
            root_node = classifier_roots[i]
            alpha_current = classifier_output[i]

            node_label = predict(root_node, row)
            if node_label == 0:
                sum_classifiers = sum_classifiers - alpha_current
            else:
                sum_classifiers = sum_classifiers + alpha_current
        
        if sum_classifiers > 0:
            current_prediction = 1
        else:
            current_prediction = 0
        
        predicted_results.append(current_prediction)
    
    #calculate result measure for current fold
    
    fold_result = calc_results(predicted_results, test_dataset)
    
    #append fold result in the final list
    accuracies.append(fold_result[0])
    precisions.append(fold_result[1])
    recalls.append(fold_result[2])
    f1_measures.append(fold_result[3])
    
    #print results of each fold
    print("Fold accuracy::" + str(fold_result[0]))
    print("Fold precision::" + str(fold_result[1]))
    print("Fold recall::" + str(fold_result[2]))
    print("Fold f1 measure::" + str(fold_result[3]))

# final combination of the output from all the n_folds
sum_alpha = 0

#after all the folds- find average of all the fold
resultant_accuracy = sum(accuracies)/ len(accuracies)
resultant_precision = sum(precisions)/ len(precisions)
resultant_recall = sum(recalls)/ len(recalls)
resultant_f1measure = sum(f1_measures)/len(f1_measures)
print("------------------------------------------------")
print("Final accuracy::" + str(resultant_accuracy))
print("Final precision::" + str(resultant_precision))
print("Final recall::" + str(resultant_recall))
print("Final f1::" + str(resultant_f1measure))

----------------beginnning of a fold---------------------
Fold accuracy::0.6521739130434783
Fold precision::0.7
Fold recall::0.23333333333333334
Fold f1 measure::0.35
----------------beginnning of a fold---------------------
Fold accuracy::0.7391304347826086
Fold precision::0.6153846153846154
Fold recall::0.23529411764705882
Fold f1 measure::0.3404255319148936
----------------beginnning of a fold---------------------
Fold accuracy::0.6304347826086957
Fold precision::0.6666666666666666
Fold recall::0.20689655172413793
Fold f1 measure::0.3157894736842105
----------------beginnning of a fold---------------------
Fold accuracy::0.6304347826086957
Fold precision::0.5263157894736842
Fold recall::0.3448275862068966
Fold f1 measure::0.4166666666666667
----------------beginnning of a fold---------------------
Fold accuracy::0.6521739130434783
Fold precision::1.0
Fold recall::0.03333333333333333
Fold f1 measure::0.06451612903225806
----------------beginnning of a fold---------------------
Fold a